In [1]:
!pip install pyspark==3.5.0 findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=a63835c2caec0063c300423df03cade9dfa1fbdd65bb592e2ad18e6ac234acc3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Day4").getOrCreate()

In [32]:
 movies_df = spark.read.csv("sample_data/movies_metadata.csv", header=True, inferSchema=True)

 movies_df.show(5)



+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------+--------------------+-----------------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|  status|             tagline|               title|   video|        vote_average|       vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+-----

In [16]:
movies_df.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [17]:
movies_df.describe().show()

+-------+--------------------+---------------------+--------------------+--------------------+--------------------+--------------------+---------+------------------+--------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|summary|               adult|belongs_to_collection|              budget|              genres|            homepage|                  id|  imdb_id| original_language|original_title|         overview|         popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|              status|             tagline|    title|               video|        vote_average|          vote_count|
+-------+--------------------+--------

In [33]:
ratings_df = movies_df.select("title", "vote_average", "release_date")
ratings_df.show()

+--------------------+--------------------+--------------------+
|               title|        vote_average|        release_date|
+--------------------+--------------------+--------------------+
|           Toy Story|                 7.7|          1995-10-30|
|             Jumanji|                 6.9|          1995-12-15|
|    Grumpier Old Men|                 6.5|          1995-12-22|
|[{'iso_639_1': 'e...|Friends are the p...|/16XOMpEaLWkrcPqS...|
|Father of the Bri...|                 5.7|          1995-02-10|
|                Heat|                 7.7|          1995-12-15|
|             Sabrina|                 6.2|          1995-12-15|
|        Tom and Huck|                 5.4|          1995-12-22|
|        Sudden Death|                 5.5|          1995-12-22|
|           GoldenEye|                 6.6|          1995-11-16|
|The American Pres...|                 6.5|          1995-11-17|
|Dracula: Dead and...|                 5.7|          1995-12-22|
|               Balto|   

In [34]:
action_movies = movies_df.filter(movies_df.genres.like("%Action%"))
action_movies.show(5)

high_rated_movies = movies_df.filter(movies_df.vote_average > 7.5)
high_rated_movies.show(5)


+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+----------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+----------------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|  original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|           title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+----------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+-

In [35]:
from pyspark.sql.functions import year, col
movies_with_year = movies_df.withColumn("release_year", year(movies_df.release_date))

movies_with_year.show(5)

# Example (assuming 'budget' and 'revenue' columns exist)
movies_with_profit = movies_df.withColumn("profit", col("revenue") - col("budget"))
movies_with_profit.show(5)

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------+--------------------+-----------------+------------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|        release_date|             revenue|             runtime|    spoken_languages|  status|             tagline|               title|   video|        vote_average|       vote_count|release_year|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+-----------

In [43]:
from pyspark.sql.functions import cast, avg
from pyspark.sql.types import DoubleType

avg_ratings_by_genre = movies_df.groupBy("genres") \
                        .agg(avg(col("vote_average").cast(DoubleType()))) \
                        .orderBy("avg(CAST(vote_average AS DOUBLE))", ascending=False)

avg_ratings_by_genre.show(5)



+--------------------+---------------------------------+
|              genres|avg(CAST(vote_average AS DOUBLE))|
+--------------------+---------------------------------+
|[{'id': 35, 'name...|                     2.96655431E8|
|[{'id': 18, 'name...|                     2.52276927E8|
|[{'id': 12, 'name...|             1.1615207522222223E7|
|[{'id': 10402, 'n...|                       1154056.08|
|[{'id': 28, 'name...|                798873.9236363636|
+--------------------+---------------------------------+
only showing top 5 rows



In [55]:
movies_with_year.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu